In [119]:
from blackbox37 import check
import string
import random

available_characters = list(string.ascii_uppercase) + [str(i) for i in range(10)]
available_characters.append("")
GROUP_ID = 2

In [120]:
print(available_characters)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '']


In [125]:
def generate():
#     size = random.randint(12, 18)
    size = 18
    individual = random.choices(available_characters, k=size)
    while len(individual) < 12:
        individual = random.choices(available_characters, k=size)
    return individual

In [126]:
sol = ''.join(generate())
print(sol)
print(check(group_number=2, attempt=sol))

SDV01S7AUHHPX4GQV
0.44975490196078427


# Population definition

In [144]:
def evaluate(x):
    x['score'] = check(GROUP_ID, x['phenotype'])
    return x

def generate_population(size):
    return [{'phenotype': generate(), 'score': None} for i in range(size)]


class RandomExploration:
    def evolve(self, pop):
        pop.population = generate_population(len(pop.population))

        
class Population:
    def __init__(self, size, exploration=RandomExploration()):
        self.size = size
        self.population = generate_population(size)
        self.exploration = exploration
        
    def evaluate(self):
        self.population = sorted(map(evaluate, self.population), key=lambda x: x['score'], reverse=True)
        
    def evolve(self):
        self.exploration.evolve(self)
    
    def __str__(self):
        return ''.join(map(lambda x: ''.join(x['phenotype']) + f" ({x['score']})\n", self.population)).strip()

### Testing Population

In [128]:
test_pop = Population(3)
test_pop.evaluate()
print(str(test_pop))

A1UZQDT1C1G5OH77UW (0.5220458553791888)
UAMBJKWR1KM4RZLQQ6 (0.4772985705003248)
T9UV7AA52ZYVSKU73 (0.3077731092436975)


# Stratégies d'exploration

In [142]:
def evolve(pop, epochs=5):
    for epoch in range(epochs):
        pop.evaluate()
        print("------------------------")
        print(f"Epoch {epoch+1}/{epochs}")
        print(str(pop))
        pop.evolve()

In [143]:
population1 = Population(5)
evolve(population1)

------------------------
Epoch 1/5
1EQKWBNUEO0J3NCLO (0.4935383244206774)
J3CFFMYB9B2XJKXWK (0.49264705882352944)
DLV0QGUQ5FZ5IMBSTL (0.4820750237416905)
BBOZZFMJM9VBSL7YO (0.44975490196078427)
XPWNWDMV82T9S8LVH (0.43156604747162025)
------------------------
Epoch 2/5
AX3I3EF0EM5WRG9OWY (0.5012654320987654)
V7N6JL67PW3YDCR1 (0.47222222222222227)
UZG7KMEIPLDWG8UZD7 (0.4611178207192701)
SFYLKO20GLMOGWSOC (0.447730179028133)
VHAMIACXZPCWRDBIL8 (0.44009539842873185)
------------------------
Epoch 3/5
5597LLQBON9NXOIDSD (0.4680918663761801)
MWBED9RSP8KJQGDRVS (0.46593915343915343)
8V7K61MB9EVEB1A0UV (0.445679012345679)
F38TSUB2J0RCDP0LK (0.4355392156862745)
F43A1G5QT113SYPR (0.3680555555555556)
------------------------
Epoch 4/5
CH36F49HHGSMUUG6RR (0.5397673314339982)
L6GBWIXOY3D1H1KC5T (0.5231481481481481)
6DFPUVG285OM2A0STC (0.43937869028448745)
BG9PP558DTBNL0K5 (0.40833333333333344)
6PUXK5JBD42VKAWI (0.3993055555555556)
------------------------
Epoch 5/5
3U7J94FWXSWMHCMKM4 (0.49567901234

In [146]:
def selection(pop, size_bests):
    return pop.population[:size_bests]

In [151]:
# class CrossOver:
    
class CrossOverRandomTwo:
    def __init__(self, prob):
        self.prob = prob
        
    def __call__(self, bests):
        indiv = random.sample(bests, k=2)
        rand = random.randint(0, 18)
        return {'genotype': indiv[0]['genotype'][:rand] + indiv[1]['genotype'][rand:], 'score': None}

In [152]:
class SimpleExploration:
    def __init__(self, cross_over, mutation, bests_size=5):
        self.cross_over = cross_over
        self.cross_over = cross_over
        self.mutate = mutation
        self.bests_size
        
    def evolve(pop):
        new_population = []
        # Get bests individuals
        bests = selection(pop, n=self.bests_size)
        # Do mutations for the rest of the population
        for i in range(pop.size - self.bests_size): ########################## FIXME Gérer les individus "mort-nés"
            if random.random() < self.cross_over.prob:
                individual = cross_over(bests)
            else:
                individual = bests[0] ############################################# FIXME Faire une copie de l'individu
            
            self.mutate(individual) # In-place
            new_population.append(individual)
        pop.population = new_population